In [1]:
"""Downloads XML files from the Bundestag API, starting from the Merkel IV period (2017)."""

import requests

API_KEY = "yyYKvVp.XtikEvPb5uU6PrWQHnqCU5lJLQY2rPOKF9" #sowas gehört eigentlich in eine separate file

ressourcentyp = "plenarprotokoll-text" #@Anni, falls das der falsche Ressourcentyp ist, kannst du es hier ändern
datum_start = "2017-10-26"
datum_end = "2022-03-01"

api_call_xml = f"https://search.dip.bundestag.de/api/v1/{ressourcentyp}/?f.datum.start={datum_start}&f.datum.end={datum_end}&apikey={API_KEY}&format=xml"
data_xml = requests.get(api_call_xml)
print(data_xml.status_code)

200


In [10]:
""" Variante mit dem Python Wrapper "BundestagsAPy". Den installiert man via pip install git+https://github.com/parobo/BundestagsAPy (global kann das nur Kiril/ Nico?)."""

import BundestagsAPy
client = BundestagsAPy.Client(API_KEY)
results = client.bt_plenarprotokoll_text(max_results=False, start_date="2017-10-26", end_date="2022-03-01")
print(len(results))

317
